# ATIS Flight Reservations Dataset

Dataset download link: http://lisaweb.iro.umontreal.ca/transfert/lisa/users/mesnilgr/atis/




## Understanding the Data

In [690]:
import numpy as np
import pandas as pd
import nltk, pprint, os
import gzip, os, pickle
import matplotlib.pyplot as plt
import random

In [691]:
# read the first part of the dataset
# each part (.gz file) contains train, validation and test sets, plus a dict

filename = 'atis.fold0.pkl.gz'
f = gzip.open(filename, 'rb')
try:
    train_set, valid_set, test_set, dicts = pickle.load(f, encoding='latin1')
except:
    train_set, valid_set, test_set, dicts = pickle.load(f)
finally:
    f.close()


In [692]:
np.shape(train_set)

(3, 3983)

In [693]:
# structure of the component data files
print(np.shape(train_set))
print(np.shape(valid_set))
print(np.shape(test_set))

(3, 3983)
(3, 995)
(3, 893)


In [694]:
# each set is a 3-tuple, each element of the tuple being a list 
print(len(train_set))
print(type(train_set[0]))
print(len(train_set[0]))


3
<class 'list'>
3983


The first list has 3983 arrays, each array being a sentence. The words are encoded by numbers (and have to be decoded using the dict provided).

Let's store the three lists into separate objects.

In [695]:
# storing the three elements of the tuple in three objects
train_x, _, train_label = train_set
val_x, _, val_label = valid_set
test_x, _, test_label = test_set

The first list represents the actual words (encoded), and the third list contains their labels (again, encoded).

In [696]:
# each list in the tuple is a numpy array (a sentence)
# printing first list in the tuple's first element
train_x[0]

array([554, 194, 268,  64,  62,  16,   8, 234, 481,  20,  40,  58, 234,
       415, 205])

In [697]:
# labels are stored in the third list train_label
train_label[0]

array([126, 126, 126,  48, 126,  36,  35, 126, 126,  33, 126, 126, 126,
        78, 123])

In [698]:
# dicts 
print(type(dicts))
print(dicts.keys())

<class 'dict'>
dict_keys(['labels2idx', 'tables2idx', 'words2idx'])


In [699]:
# each key:value pair is itself a dict
print(type(dicts['labels2idx']))
print(type(dicts['tables2idx']))
print(type(dicts['words2idx']))


<class 'dict'>
<class 'dict'>
<class 'dict'>


In [700]:
# storing labels and words in separate variables
words = dicts['words2idx']
labels = dicts['labels2idx']
tables = dicts['tables2idx']

In [701]:
# each key of words_dict is a word, each value its index
words.keys()

dict_keys(['all', 'coach', 'cincinnati', 'people', 'month', 'four', 'code', 'go', 'show', 'thursday', 'to', 'restriction', 'dinnertime', 'under', 'sorry', 'include', 'midwest', 'worth', 'southwest', 'me', 'returning', 'far', 'vegas', 'airfare', 'ticket', 'difference', 'arrange', 'tickets', 'louis', 'cheapest', 'list', 'wednesday', 'leave', 'heading', 'ten', 'direct', 'turboprop', 'rate', 'cost', 'quebec', 'layover', 'air', 'what', 'stands', 'chicago', 'schedule', 'transcontinental', 'goes', 'new', 'transportation', 'here', 'hours', 'let', 'twentieth', 'along', 'thrift', 'passengers', 'great', 'thirty', 'canadian', 'leaves', 'alaska', 'leaving', 'amount', 'weekday', 'makes', 'midway', 'montreal', 'via', 'depart', 'county', 'names', 'stand', 'total', 'seventeenth', 'use', 'twa', 'from', 'would', 'abbreviations', 'destination', 'only', 'next', 'live', 'shortest', 'limousine', 'tell', 'today', 'more', 'DIGIT', 'm80', 'downtown', 'train', 'tampa', 'fly', 'f', 'this', 'car', 'anywhere', 'can

In [702]:
# now, we can map the numeric values v in a sentence with the k,v in the dict
# train_x contains the list of training sentences
# this is the first sentence
[k for val in train_x[0] for k,v in words.items() if v==val]

['what',
 'flights',
 'leave',
 'atlanta',
 'at',
 'about',
 'DIGIT',
 'in',
 'the',
 'afternoon',
 'and',
 'arrive',
 'in',
 'san',
 'francisco']

In [703]:
# let's look at the first few sentences
sents = []
for i in range(30):
    sents.append(' '.join([k for val in train_x[i] for k,v in words.items() if v==val]))

sents

['what flights leave atlanta at about DIGIT in the afternoon and arrive in san francisco',
 'what is the abbreviation for canadian airlines international',
 "i 'd like to know the earliest flight from boston to atlanta",
 'show me the us air flights from atlanta to boston',
 'show me the cheapest round trips from dallas to baltimore',
 "i 'd like to see all flights from denver to philadelphia",
 'explain fare code qx',
 "i 'd like a united airlines flight on wednesday from san francisco to boston",
 'what is the price of american airlines flight DIGITDIGIT from new york to los angeles',
 'what does the meal code s stand for',
 'what are all flights to denver from philadelphia on sunday',
 'what times does the late afternoon flight leave from washington for denver',
 'what flights are available monday from san francisco to pittsburgh',
 'what airlines have business class',
 'flights from atlanta to washington dc',
 'from new york to toronto on thursday morning',
 'show me all the direct

In [704]:
# labels dict contains IOB (inside-out-beginning) labelled entities
labels.keys()

dict_keys(['B-time_relative', 'B-stoploc.state_code', 'B-depart_date.today_relative', 'B-arrive_date.date_relative', 'B-depart_date.date_relative', 'I-restriction_code', 'B-return_date.month_name', 'I-time', 'B-depart_date.day_name', 'I-arrive_time.end_time', 'B-fromloc.airport_code', 'B-cost_relative', 'B-connect', 'B-return_time.period_mod', 'B-arrive_time.period_mod', 'B-flight_number', 'B-depart_time.time_relative', 'I-toloc.city_name', 'B-arrive_time.period_of_day', 'B-depart_time.period_of_day', 'I-return_date.date_relative', 'I-depart_time.start_time', 'B-fare_amount', 'I-depart_time.time_relative', 'B-city_name', 'B-depart_date.day_number', 'I-meal_description', 'I-depart_date.today_relative', 'I-airport_name', 'I-arrive_date.day_number', 'B-toloc.state_code', 'B-arrive_date.month_name', 'B-stoploc.airport_code', 'I-depart_time.time', 'B-airport_code', 'B-arrive_time.start_time', 'B-period_of_day', 'B-arrive_time.time', 'I-flight_stop', 'B-toloc.state_name', 'B-booking_class', 

There are 127 classes of labels (including the 'O' - tokens that do not fall into any entity).

In [705]:
# number of labels
print(len(labels.keys()))

127


Since the dicts 'words' and 'labels' are key:value pairs of index:word/label, let's reverse the dicts so that we don't have to do a reverse lookup everytime.

In [706]:
# converting words_to_id to id_to_words
# and labels_to_id to id_to_labels
id_to_words = {words[k]:k for k in words}
id_to_labels = {labels[k]:k for k in labels}

Now we can print the words and corresponding labels simply by looking up the value of a numeric index of each word, for e.g.:

In [707]:
# printing a few randomly chosen sentences and the corresponding labels (tagged entities)
for i in random.sample(range(len(train_x)), 20):
    w = list(map(lambda x: id_to_words[x], train_x[i]))
    l = list(map(lambda x: id_to_labels[x], train_label[i]))
    print(list(zip(w, l)))
    print('\n')

[('i', 'O'), ("'d", 'O'), ('like', 'O'), ('to', 'O'), ('find', 'O'), ('a', 'O'), ('nonstop', 'B-flight_stop'), ('flight', 'O'), ('from', 'O'), ('boston', 'B-fromloc.city_name'), ('to', 'O'), ('atlanta', 'B-toloc.city_name'), ('that', 'O'), ('leaves', 'O'), ('sometime', 'O'), ('in', 'O'), ('the', 'O'), ('afternoon', 'B-arrive_time.period_of_day'), ('and', 'O'), ('arrives', 'O'), ('in', 'O'), ('atlanta', 'B-toloc.city_name'), ('before', 'B-arrive_time.time_relative'), ('evening', 'B-arrive_time.time')]


[('airline', 'O'), ('and', 'O'), ('flight', 'O'), ('number', 'O'), ('from', 'O'), ('columbus', 'B-fromloc.city_name'), ('to', 'O'), ('minneapolis', 'B-toloc.city_name')]


[('what', 'O'), ('is', 'O'), ('the', 'O'), ('latest', 'B-flight_mod'), ('flight', 'O'), ('leaving', 'O'), ('washington', 'B-fromloc.city_name'), ('for', 'O'), ('denver', 'B-toloc.city_name')]


[('give', 'O'), ('me', 'O'), ('nonstop', 'B-flight_stop'), ('flights', 'O'), ('from', 'O'), ('new', 'B-fromloc.city_name'), ('

Let's write a function which takes in an index and returns the corresponding query with its labels.

In [708]:
def print_query(index):
    w = list(map(lambda x: id_to_words[x], train_x[index]))
    l = list(map(lambda x: id_to_labels[x], train_label[index]))
    s = list(zip(w, l))
    return s

In [709]:
print_query(3925)

[('on', 'O'),
 ('<UNK>', 'B-airline_name'),
 ('air', 'I-airline_name'),
 ('how', 'O'),
 ('many', 'O'),
 ('flights', 'O'),
 ('leaving', 'O'),
 ('oakland', 'B-fromloc.city_name'),
 ('on', 'O'),
 ('july', 'B-depart_date.month_name'),
 ('twenty', 'B-depart_date.day_number'),
 ('seventh', 'I-depart_date.day_number'),
 ('to', 'O'),
 ('boston', 'B-toloc.city_name'),
 ('nonstop', 'B-flight_stop')]

Also, some queries specify stopover cities, such as this.

In [710]:
print_query(3443)

[('is', 'O'),
 ('there', 'O'),
 ('a', 'O'),
 ('flight', 'O'),
 ('between', 'O'),
 ('oakland', 'B-fromloc.city_name'),
 ('and', 'O'),
 ('boston', 'B-toloc.city_name'),
 ('with', 'O'),
 ('a', 'O'),
 ('stopover', 'O'),
 ('in', 'O'),
 ('dallas', 'B-stoploc.city_name'),
 ('fort', 'I-stoploc.city_name'),
 ('worth', 'I-stoploc.city_name'),
 ('on', 'O'),
 ('twa', 'B-airline_code')]

We can see that in this dataset, queries are far more complex (in terms of number of labels, variety in the sentence structures etc.) and thus we cannot  write simple hand-written rules to extract chunks such as to_from_city, types_of_meals etc. 

Thus, we need to train probabilistic models such as CRFs, HMMs etc. to tag each word with its corresponding entity label.

We'll use the training and validation sets ```train_x``` and ```valid_x``` as to tune the model, and finaly use test set to measure the performance.

## Models for NER

Let's experiment with a few different models for labelling words with named entities.


In [711]:
# POS tagging sentences
# takes in a list of sentences and returns a list of POS-tagged sentences
# in the form (word, tag)

def pos_tag(sent_list):
    pos_tags = []    
    for sent in sent_list:
        tagged_words = nltk.pos_tag([id_to_words[val] for val in sent])
        pos_tags.append(tagged_words)
    return pos_tags

In [712]:
# pos tagging train, validation and test sets
train_pos = pos_tag(train_x)
valid_pos = pos_tag(val_x)
test_pos = pos_tag(test_x)

In [713]:
# looking at tags of some randomly chosen queries
# notice that most cities after 'TO' are tagged as VB
i = random.randrange(len(train_pos))
train_pos[i]

[('show', 'VB'),
 ('me', 'PRP'),
 ('flights', 'NNS'),
 ('from', 'IN'),
 ('denver', 'NN'),
 ('to', 'TO'),
 ('philadelphia', 'VB')]

To train a model, we need the entity labels of each word along with the POS tags, for e.g. in this format:
```[('New', 'NNP', u'B-GPE'), ('York', 'NNP', u'I-GPE'), ('is', 'VBZ', u'O'), ('my', 'PRP$', u'O'), ('favorite', 'JJ', u'O'), ('city', 'NN', u'O')]```

Let's convert the training and validation sentences to this form. 

In [714]:
# function to create (word, pos_tag, iob_label) tuples for a given dataset
def create_word_pos_label(pos_tagged_data, labels):
    iob_labels = []
    for sent in list(zip(pos_tagged_data, labels)):
        pos = sent[0]
        labels = sent[1]
        l = list(zip(pos, labels))
        tuple_3 = [(i[0][0], i[0][1], id_to_labels[i[1]]) for i in l]
        iob_labels.append(tuple_3)
    return iob_labels

In [715]:
train_labels = create_word_pos_label(train_pos, train_label)
train_labels

[[('what', 'WP', 'O'),
  ('flights', 'NNS', 'O'),
  ('leave', 'VBP', 'O'),
  ('atlanta', 'VBN', 'B-fromloc.city_name'),
  ('at', 'IN', 'O'),
  ('about', 'RB', 'B-depart_time.time_relative'),
  ('DIGIT', 'NNP', 'B-depart_time.time'),
  ('in', 'IN', 'O'),
  ('the', 'DT', 'O'),
  ('afternoon', 'NN', 'B-depart_time.period_of_day'),
  ('and', 'CC', 'O'),
  ('arrive', 'NN', 'O'),
  ('in', 'IN', 'O'),
  ('san', 'JJ', 'B-toloc.city_name'),
  ('francisco', 'NN', 'I-toloc.city_name')],
 [('what', 'WP', 'O'),
  ('is', 'VBZ', 'O'),
  ('the', 'DT', 'O'),
  ('abbreviation', 'NN', 'O'),
  ('for', 'IN', 'O'),
  ('canadian', 'JJ', 'B-airline_name'),
  ('airlines', 'NNS', 'I-airline_name'),
  ('international', 'JJ', 'I-airline_name')],
 [('i', 'JJ', 'O'),
  ("'d", 'MD', 'O'),
  ('like', 'VB', 'O'),
  ('to', 'TO', 'O'),
  ('know', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('earliest', 'JJS', 'B-flight_mod'),
  ('flight', 'NN', 'O'),
  ('from', 'IN', 'O'),
  ('boston', 'NN', 'B-fromloc.city_name'),
  ('to', 'TO

In [716]:
# some sample training sentences
train_labels[random.randrange(len(train_labels))]

[('what', 'WP', 'O'),
 ('flights', 'NNS', 'O'),
 ('are', 'VBP', 'O'),
 ('there', 'RB', 'O'),
 ('from', 'IN', 'O'),
 ('new', 'JJ', 'B-fromloc.city_name'),
 ('york', 'NN', 'I-fromloc.city_name'),
 ('city', 'NN', 'I-fromloc.city_name'),
 ('to', 'TO', 'O'),
 ('las', 'VB', 'B-toloc.city_name'),
 ('vegas', 'NN', 'I-toloc.city_name')]

In [717]:
# doing the same for validation and test data
valid_labels = create_word_pos_label(valid_pos, val_label)
test_labels = create_word_pos_label(test_pos, test_label)

### Converting to Tree Format

Let's now convert the sentences into a tree format, which is needed by NLTK to train taggers.

In [718]:
from nltk.corpus import conll2000
from nltk import conlltags2tree, tree2conlltags

# converting a sample sentence to a tree
tree = conlltags2tree(train_labels[2])
print(tree)

(S
  i/JJ
  'd/MD
  like/VB
  to/TO
  know/VB
  the/DT
  (flight_mod earliest/JJS)
  flight/NN
  from/IN
  (fromloc.city_name boston/NN)
  to/TO
  (toloc.city_name atlanta/VB))


Let's now convert all training sentences to trees.

In [719]:
# converting training, validation and test datasets to tree format
train_trees = [conlltags2tree(sent) for sent in train_labels]
valid_trees = [conlltags2tree(sent) for sent in valid_labels]
test_trees = [conlltags2tree(sent) for sent in test_labels]

In [720]:
# print some sample training trees
print(train_trees[random.randrange(len(train_trees))])

(S
  what/WP
  is/VBZ
  the/DT
  (cost_relative cheapest/JJS)
  fare/NN
  from/IN
  (fromloc.city_name dallas/NNS)
  to/TO
  (toloc.city_name denver/VB)
  (round_trip round/NN trip/NN))


Let's now try building some parsers. 

### Regex Based Parsers

Let's start with a dummy parser - one which tags every token as an 'O'.

In [721]:
# a dummy chunk parser - tags every word as 'O'
cp = nltk.RegexpParser(r'')
print(cp.evaluate(valid_trees))

ChunkParse score:
    IOB Accuracy:  63.6%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%


The above results tell us that about 63% of the tokens are tagged as 'O', i.e. they are not a named entity of any type. The precision, recall etc. are zero because we did not find any chunks at all.

### Unigram Chunker

Let's now try a unigram chunker.

In [722]:
# unigram chunker

from nltk import ChunkParserI

class UnigramChunker(ChunkParserI):    
    def __init__(self, train_sents):
        # convert train sents from tree format to tags
        train_data = [[(t, c) for w, t, c in nltk.chunk.tree2conlltags(sent)] 
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)
        
    def parse(self, sentence):
        pos_tags = [pos for (word, pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        
        # convert to tree again
        conlltags = [(word, pos, chunktag) for ((word, pos), chunktag) in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
        

In [723]:
# unigram chunker 
unigram_chunker = UnigramChunker(train_trees)
print(unigram_chunker.evaluate(valid_trees))

ChunkParse score:
    IOB Accuracy:  66.3%%
    Precision:     37.5%%
    Recall:        18.5%%
    F-Measure:     24.8%%


The accuracy, precision and recall have of course improved compared to the previous dummy parser. Let's also look at what the unigram parser has learnt.

In [724]:
# printing the most likely IOB tags for each POS tag

# extract the list of pos tags
postags = sorted(set([pos for sent in train_trees for (word, pos) in sent.leaves()]))

# for each tag, assign the most likely IOB label
print(unigram_chunker.tagger.tag(postags))

[('CC', 'O'), ('CD', 'B-round_trip'), ('DT', 'O'), ('EX', 'O'), ('FW', 'B-fromloc.city_name'), ('IN', 'O'), ('JJ', 'O'), ('JJR', 'B-cost_relative'), ('JJS', 'B-cost_relative'), ('MD', 'O'), ('NN', 'O'), ('NNP', 'B-depart_time.time'), ('NNS', 'O'), ('PDT', 'O'), ('POS', 'O'), ('PRP', 'O'), ('PRP$', 'O'), ('RB', 'O'), ('RBR', 'B-cost_relative'), ('RBS', 'B-cost_relative'), ('RP', 'O'), ('TO', 'O'), ('VB', 'B-toloc.city_name'), ('VBD', 'O'), ('VBG', 'O'), ('VBN', 'O'), ('VBP', 'O'), ('VBZ', 'O'), ('WDT', 'O'), ('WP', 'O'), ('WRB', 'O')]


The unigram tagger has learnt that most pos tags are indeed an 'O', i.e. don't form an entity. Some interesting patterns it has learnt are:
- JJR, JJS (relative adjectives), are most likely B-cost_relative (e.g. cheapest, cheaper)
- NNP is most likely to be B-depart_time.time

### Bigram Chunker

Let's try a bigram chunker as well - we just need to change the ```UnigramTagger``` to ```BigramTagger```.

In [725]:
# bigram tagger

class BigramChunker(ChunkParserI):    
    def __init__(self, train_sents):
        # convert train sents from tree format to tags
        train_data = [[(t, c) for w, t, c in nltk.chunk.tree2conlltags(sent)] 
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)
        
    def parse(self, sentence):
        pos_tags = [pos for (word, pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        
        # convert to tree again
        conlltags = [(word, pos, chunktag) for ((word, pos), chunktag) in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
        

In [726]:
# unigram chunker 
bigram_chunker = BigramChunker(train_trees)
print(bigram_chunker.evaluate(valid_trees))

ChunkParse score:
    IOB Accuracy:  70.6%%
    Precision:     43.5%%
    Recall:        38.8%%
    F-Measure:     41.0%%


The metrics have improved significantly from unigram to bigram.

## Classifier Based Chunkers



In [727]:
class ConsecutiveNPChunkTagger(nltk.TaggerI): 

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history) 
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI): 
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [728]:
# extracts features for a given word i in a given sentence 
# history refers to the previous POS tags in the sentence
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    
    # the first word has both previous word and previous tag undefined
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]

    # gazetteer lookup features (see section below)
    gazetteer = gazetteer_lookup(word)

    return {"pos": pos, "prevpos": prevpos, 'word':word,
           'word_is_city': gazetteer[0],
           'word_is_state': gazetteer[1],
           'word_is_county': gazetteer[2]}

In [729]:
# example features for a given sentence
sent_pos = train_pos[0]
sent_pos

[('what', 'WP'),
 ('flights', 'NNS'),
 ('leave', 'VBP'),
 ('atlanta', 'VBN'),
 ('at', 'IN'),
 ('about', 'RB'),
 ('DIGIT', 'NNP'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('afternoon', 'NN'),
 ('and', 'CC'),
 ('arrive', 'NN'),
 ('in', 'IN'),
 ('san', 'JJ'),
 ('francisco', 'NN')]

In [730]:
# example features for a sentence
for i in range(len(sent_pos)):
    print(npchunk_features(sent_pos, i, history=[]))
    print(' ')

{'pos': 'WP', 'prevpos': '<START>', 'word': 'what', 'word_is_city': False, 'word_is_state': False, 'word_is_county': False}
 
{'pos': 'NNS', 'prevpos': 'WP', 'word': 'flights', 'word_is_city': False, 'word_is_state': False, 'word_is_county': False}
 
{'pos': 'VBP', 'prevpos': 'NNS', 'word': 'leave', 'word_is_city': False, 'word_is_state': False, 'word_is_county': False}
 
{'pos': 'VBN', 'prevpos': 'VBP', 'word': 'atlanta', 'word_is_city': True, 'word_is_state': False, 'word_is_county': False}
 
{'pos': 'IN', 'prevpos': 'VBN', 'word': 'at', 'word_is_city': False, 'word_is_state': False, 'word_is_county': False}
 
{'pos': 'RB', 'prevpos': 'IN', 'word': 'about', 'word_is_city': False, 'word_is_state': False, 'word_is_county': False}
 
{'pos': 'NNP', 'prevpos': 'RB', 'word': 'DIGIT', 'word_is_city': False, 'word_is_state': False, 'word_is_county': False}
 
{'pos': 'IN', 'prevpos': 'NNP', 'word': 'in', 'word_is_city': False, 'word_is_state': False, 'word_is_county': False}
 
{'pos': 'DT', '

In [731]:
# training the chunker 
chunker = ConsecutiveNPChunker(train_trees)

In [732]:
# evaluate the chunker
print(chunker.evaluate(valid_trees))

ChunkParse score:
    IOB Accuracy:  91.7%%
    Precision:     75.3%%
    Recall:        81.8%%
    F-Measure:     78.4%%


The results have improved significantly compared to the basic unigram/bigram chunkers, and they may improve further if we create better features.

For example, if the word is 'DIGIT' (numbers are labelled as 'DIGIT' in this dataset), we can have a feature which indicates that (see example below). In this dataset, 4-digit numbers are encoded as 'DIGITDIGITDIGITDIGIT'.

In [733]:
# example of 'DIGITDIGIT'
train_pos[1326]

[('do', 'VBP'),
 ('you', 'PRP'),
 ('have', 'VB'),
 ('an', 'DT'),
 ('DIGITDIGITDIGIT', 'NNP'),
 ('flight', 'NN'),
 ('from', 'IN'),
 ('denver', 'NN'),
 ('to', 'TO'),
 ('san', 'VB'),
 ('francisco', 'NN')]

Let's add some of these features and see if the performance improves.

In [734]:
# extracts features for a given word i in a given sentence 
# history refers to the previous POS tags in the sentence
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    
    # the first word has both previous word and previous tag undefined
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
        
    if i == len(sentence)-1:
        nextword, nextpos = '<END>', '<END>'
    else:
        nextword, nextpos = sentence[i+1]

    # gazetteer lookup features (see section below)
    gazetteer = gazetteer_lookup(word)

    # adding word_is_digit feature (boolean)
    return {"pos": pos, "prevpos": prevpos, 'word':word, 
           'word_is_city': gazetteer[0],
           'word_is_state': gazetteer[1],
           'word_is_county': gazetteer[2],
           'word_is_digit': word in 'DIGITDIGITDIGIT', 
           'nextword': nextword, 
           'nextpos': nextpos}

In [735]:
# train and evaluate the chunker 
chunker = ConsecutiveNPChunker(train_trees)
print(chunker.evaluate(valid_trees))

ChunkParse score:
    IOB Accuracy:  91.7%%
    Precision:     75.9%%
    Recall:        85.1%%
    F-Measure:     80.3%%


In [736]:

# ChunkParse score:
#     IOB Accuracy:  92.7%%
#     Precision:     78.1%%
#     Recall:        84.9%%
#     F-Measure:     81.4%%

# ChunkParse score:
#     IOB Accuracy:  91.7%%
#     Precision:     75.5%%
#     Recall:        82.0%%
#     F-Measure:     78.6%%

### Using a Gazetteer to Lookup Cities and States

URL: https://raw.githubusercontent.com/grammakov/USA-cities-and-states/master/us_cities_states_counties.csv

In [737]:
# reading a file containing list of US cities, states and counties
us_cities = pd.read_csv("us_cities_states_counties.csv", sep="|")
us_cities.head()


,City,State short,State full,County,City alias
0,Holtsville,NY,New York,SUFFOLK,Internal Revenue Service
1,Holtsville,NY,New York,SUFFOLK,Holtsville
2,Adjuntas,PR,Puerto Rico,ADJUNTAS,URB San Joaquin
3,Adjuntas,PR,Puerto Rico,ADJUNTAS,Jard De Adjuntas
4,Adjuntas,PR,Puerto Rico,ADJUNTAS,Colinas Del Gigante


In [738]:
# storing cities, states and counties as sets
cities = set(us_cities['City'].str.lower())
states = set(us_cities['State full'].str.lower())
counties = set(us_cities['County'].str.lower())

In [739]:
print(len(cities))
print(len(states))
print(len(counties))

18854
62
1932


In [740]:
# define a function to look up a given word in cities, states, county
def gazetteer_lookup(word):
    return (word in cities, word in states, word in counties)

In [741]:
# sample lookups
print(gazetteer_lookup('washington'))
print(gazetteer_lookup('utah'))
print(gazetteer_lookup('philadelphia'))


(True, True, True)
(False, True, True)
(True, False, True)


### CRF Based Taggers


In [742]:
from itertools import chain
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

0.19.1


In [743]:
# structure of train/validation data
train_labels[0]

[('what', 'WP', 'O'),
 ('flights', 'NNS', 'O'),
 ('leave', 'VBP', 'O'),
 ('atlanta', 'VBN', 'B-fromloc.city_name'),
 ('at', 'IN', 'O'),
 ('about', 'RB', 'B-depart_time.time_relative'),
 ('DIGIT', 'NNP', 'B-depart_time.time'),
 ('in', 'IN', 'O'),
 ('the', 'DT', 'O'),
 ('afternoon', 'NN', 'B-depart_time.period_of_day'),
 ('and', 'CC', 'O'),
 ('arrive', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('san', 'JJ', 'B-toloc.city_name'),
 ('francisco', 'NN', 'I-toloc.city_name')]

Let's define a function to extract features from a given sentence. This is similar to the ```npchunk_features()``` function defined above, but we'll add some new features as well such as the suffix of the word (upto the last 4 characters), prefix (upto first 4 characters) etc.

The list of features we'll extract is as follows:
```
{
            'word':word,
            'pos': pos, 
            'prevword': prevword,
            'prevpos': prevpos,  
            'nextword': nextword, 
            'nextpos': nextpos,
            'word_is_city': gazetteer[0],
            'word_is_state': gazetteer[1],
            'word_is_county': gazetteer[2],
            'word_is_digit': word in 'DIGITDIGITDIGIT',
            'suff_1': suff_1,  
            'suff_2': suff_2,  
            'suff_3': suff_3,  
            'suff_4': suff_4, 
            'pref_1': pref_1,  
            'pref_2': pref_2,  
            'pref_3': pref_3, 
            'pref_4': pref_4 

}
```



In [744]:
## other features to consider

# airline code
# airline name
# day name (monday/tuesday etc.) i=1847, 2769
# o'clock (word shape): i=379

# i=random.randrange(len(train_labels))
# train_labels[i]

In [745]:
# extract features from a given sentence
def word_features(sent, i):
    word = sent[i][0]
    pos = sent[i][1]
    
    # first word
    if i==0:
        prevword = '<START>'
        prevpos = '<START>'
    else:
        prevword = sent[i-1][0]
        prevpos = sent[i-1][1]
    
    # last word
    if i == len(sent)-1:
        nextword = '<END>'
        nextpos = '<END>'
    else:
        nextword = sent[i+1][0]
        nextpos = sent[i+1][1]
    
    # word is in gazetteer
    gazetteer = gazetteer_lookup(word)
    
    # suffixes and prefixes
    pref_1, pref_2, pref_3, pref_4 = word[:1], word[:2], word[:3], word[:4]
    suff_1, suff_2, suff_3, suff_4 = word[-1:], word[-2:], word[-3:], word[-4:]
    
    return {'word':word,
            'pos': pos, 
            'prevword': prevword,
            'prevpos': prevpos,  
            'nextword': nextword, 
            'nextpos': nextpos,
            'word_is_city': gazetteer[0],
            'word_is_state': gazetteer[1],
            'word_is_county': gazetteer[2],
            'word_is_digit': word in 'DIGITDIGITDIGIT',
            'suff_1': suff_1,  
            'suff_2': suff_2,  
            'suff_3': suff_3,  
            'suff_4': suff_4, 
            'pref_1': pref_1,  
            'pref_2': pref_2,  
            'pref_3': pref_3, 
            'pref_4': pref_4 }  

In [746]:
# example features
word_features(train_labels[0], i=3)

{'nextpos': 'IN',
 'nextword': 'at',
 'pos': 'VBN',
 'pref_1': 'a',
 'pref_2': 'at',
 'pref_3': 'atl',
 'pref_4': 'atla',
 'prevpos': 'VBP',
 'prevword': 'leave',
 'suff_1': 'a',
 'suff_2': 'ta',
 'suff_3': 'nta',
 'suff_4': 'anta',
 'word': 'atlanta',
 'word_is_city': True,
 'word_is_county': False,
 'word_is_digit': False,
 'word_is_state': False}

In [747]:
# defining a few more functions to extract featrues, labels, words from sentences

def sent2features(sent):
    return [word_features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

In [748]:
# create training, validation and test sets
X_train = [sent2features(s) for s in train_labels]
y_train = [sent2labels(s) for s in train_labels]

X_valid = [sent2features(s) for s in valid_labels]
y_valid = [sent2labels(s) for s in valid_labels]

X_test = [sent2features(s) for s in test_labels]
y_test = [sent2labels(s) for s in test_labels]

In [749]:
# X_train is a list of sentences within which each feature has a corresponding dict of features
# first few words of the first sentence in X_train
X_train[0][:3]

[{'nextpos': 'NNS',
  'nextword': 'flights',
  'pos': 'WP',
  'pref_1': 'w',
  'pref_2': 'wh',
  'pref_3': 'wha',
  'pref_4': 'what',
  'prevpos': '<START>',
  'prevword': '<START>',
  'suff_1': 't',
  'suff_2': 'at',
  'suff_3': 'hat',
  'suff_4': 'what',
  'word': 'what',
  'word_is_city': False,
  'word_is_county': False,
  'word_is_digit': False,
  'word_is_state': False},
 {'nextpos': 'VBP',
  'nextword': 'leave',
  'pos': 'NNS',
  'pref_1': 'f',
  'pref_2': 'fl',
  'pref_3': 'fli',
  'pref_4': 'flig',
  'prevpos': 'WP',
  'prevword': 'what',
  'suff_1': 's',
  'suff_2': 'ts',
  'suff_3': 'hts',
  'suff_4': 'ghts',
  'word': 'flights',
  'word_is_city': False,
  'word_is_county': False,
  'word_is_digit': False,
  'word_is_state': False},
 {'nextpos': 'VBN',
  'nextword': 'atlanta',
  'pos': 'VBP',
  'pref_1': 'l',
  'pref_2': 'le',
  'pref_3': 'lea',
  'pref_4': 'leav',
  'prevpos': 'NNS',
  'prevword': 'flights',
  'suff_1': 'e',
  'suff_2': 've',
  'suff_3': 'ave',
  'suff_4': 

### Training the Model

In [750]:
# instantiate a CRF trainer from pycrfsuite
trainer = pycrfsuite.Trainer(verbose=False)

# create (word_features, word_label) pairs for every sentence
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [751]:
# Set training parameters - using L-BFGS training algorithm (default) with Elastic Net (L1 + L2) regularization.
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [752]:
# list of possible params
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [753]:
# saving the trained model to a file
trainer.train('atis.crfsuite')

### Make Predictions

In [754]:
# create a tagger object and open the trained file
tagger = pycrfsuite.Tagger()
tagger.open('atis.crfsuite')

In [755]:
# tagging a sample sentence
sample_sent = valid_labels[0]
print(' '.join(sent2tokens(sample_sent)), end='\n')

what aircraft is used on delta flight DIGITDIGITDIGITDIGIT from kansas city to salt lake city


In [756]:
print("Predicted:", ' '.join(tagger.tag(sent2features(sample_sent))))
print('\n')
print("Correct:  ", ' '.join(sent2labels(sample_sent)))

Predicted: O O O O O B-airline_name O B-flight_number O B-fromloc.city_name I-fromloc.city_name O B-toloc.city_name I-toloc.city_name I-toloc.city_name


Correct:   O O O O O B-airline_name O B-flight_number O B-fromloc.city_name I-fromloc.city_name O B-toloc.city_name I-toloc.city_name I-toloc.city_name


### Evaluating the Model

In [757]:
def iob_classification_report(y_true, y_pred):
    """
    Classification report for a list of IOB-encoded sequences.
    It computes token-level metrics and discards "O" labels.

    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    # note that we are not including 'O' as a class
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [758]:
y_pred = [tagger.tag(xseq) for xseq in X_valid]

# predictions for first sentence
y_pred[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'B-airline_name',
 'O',
 'B-flight_number',
 'O',
 'B-fromloc.city_name',
 'I-fromloc.city_name',
 'O',
 'B-toloc.city_name',
 'I-toloc.city_name',
 'I-toloc.city_name']

Let's now evaluate the model. Since we are dealing with a multiclass classification problem, we can use sklearn's ```LabelBinarizer``` to binarize labels in a one-versus-all manner.  

Read about LabelBinarizer here: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html.

In [759]:
print(iob_classification_report(y_valid, y_pred))

                              precision    recall  f1-score   support

             B-aircraft_code       1.00      0.33      0.50         3
              B-airline_code       1.00      0.93      0.96        27
              B-airline_name       1.00      0.99      1.00       139
              I-airline_name       1.00      0.97      0.99        80
              B-airport_code       0.67      0.80      0.73         5
              B-airport_name       0.50      0.43      0.46         7
              I-airport_name       0.67      0.44      0.53         9
 B-arrive_date.date_relative       0.00      0.00      0.00         1
      B-arrive_date.day_name       0.25      0.07      0.11        14
    B-arrive_date.day_number       0.50      0.12      0.19        17
    I-arrive_date.day_number       0.00      0.00      0.00         2
    B-arrive_date.month_name       0.50      0.12      0.19        17
B-arrive_date.today_relative       0.00      0.00      0.00         2
      B-arrive_time

c:\users\kshitij jain\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [760]:
# function to create (word, pos_tag, predicted_iob_label) tuples for a given dataset
def append_predicted_tags(pos_tagged_data, labels):
    iob_labels = []
    for sent in list(zip(pos_tagged_data, labels)):
        pos = sent[0]
        labels = sent[1]
        l = list(zip(pos, labels))
        tuple_3 = [(i[0][0], i[0][1], i[1]) for i in l]
        iob_labels.append(tuple_3)
    return iob_labels

In [837]:
# predictions of IOB tags on a sample validation query 
valid_tags = append_predicted_tags(valid_pos, y_pred)
valid_tags[0]

[('what', 'WP', 'O'),
 ('aircraft', 'NN', 'O'),
 ('is', 'VBZ', 'O'),
 ('used', 'VBN', 'O'),
 ('on', 'IN', 'O'),
 ('delta', 'JJ', 'O'),
 ('flight', 'NN', 'O'),
 ('DIGITDIGITDIGITDIGIT', 'NNP', 'O'),
 ('from', 'IN', 'B-fromloc.city_name'),
 ('kansas', 'NNP', 'O'),
 ('city', 'NN', 'B-toloc.city_name'),
 ('to', 'TO', 'I-toloc.city_name'),
 ('salt', 'VB', 'O'),
 ('lake', 'JJ', 'O'),
 ('city', 'NN', 'O')]

In [839]:
# create a tree using the assigned iob labels
valid_trees = [conlltags2tree(sent) for sent in valid_tags]
print(valid_trees[0])

(S
  what/WP
  aircraft/NN
  is/VBZ
  used/VBN
  on/IN
  delta/JJ
  flight/NN
  DIGITDIGITDIGITDIGIT/NNP
  (fromloc.city_name from/IN)
  kansas/NNP
  (toloc.city_name city/NN to/TO)
  salt/VB
  lake/JJ
  city/NN)


### Understanding the CRF Classifier

Let's now try to understand what the classifier has learnt. 

In [763]:
# look at the docs of tagger.info() for a list of attributes etc.
info = tagger.info()
# help(info)

In [764]:
from collections import Counter

# top 20 features, and the labels they predict
Counter(info.state_features).most_common(20)

[(('word_is_state', 'B-toloc.state_name'), 6.570057),
 (('pref_1:q', 'B-fare_basis_code'), 5.607318),
 (('prevword:round', 'I-round_trip'), 5.405693),
 (('prevword:arrive', 'B-arrive_time.time_relative'), 5.079029),
 (('word_is_state', 'B-fromloc.state_name'), 5.073412),
 (('prevword:arriving', 'B-arrive_time.time_relative'), 5.001828),
 (('prevword:restriction', 'B-restriction_code'), 4.829951),
 (('prevword:from', 'B-fromloc.city_name'), 4.730971),
 (('prevword:code', 'B-fare_basis_code'), 4.47244),
 (('prevword:between', 'B-depart_time.start_time'), 4.427766),
 (('nextword:afternoon', 'B-depart_time.period_of_day'), 4.300173),
 (('prevword:flight', 'B-flight_number'), 4.230575),
 (('pos:DT', 'O'), 4.179398),
 (('nextword:morning', 'B-depart_time.period_of_day'), 4.125129),
 (('suff_4:test', 'B-flight_mod'), 4.120814),
 (('prevword:between', 'B-arrive_time.start_time'), 3.948435),
 (('pref_1:r', 'B-round_trip'), 3.852988),
 (('pref_2:ap', 'B-restriction_code'), 3.822704),
 (('prevwor

In [765]:
# top likely transitions the model has learnt
Counter(info.transitions).most_common(15)

[(('I-today_relative', 'I-today_relative'), 7.861919),
 (('B-toloc.airport_name', 'I-toloc.airport_name'), 7.42822),
 (('I-depart_date.today_relative', 'I-depart_date.today_relative'), 7.341),
 (('B-airport_name', 'I-airport_name'), 7.249586),
 (('B-return_date.month_name', 'B-return_date.day_number'), 7.072054),
 (('B-arrive_date.month_name', 'B-arrive_date.day_number'), 6.808602),
 (('B-fromloc.state_name', 'I-fromloc.state_name'), 6.793063),
 (('B-fromloc.airport_name', 'I-fromloc.airport_name'), 6.785764),
 (('B-transport_type', 'I-transport_type'), 6.636065),
 (('B-arrive_time.period_of_day', 'I-arrive_time.period_of_day'), 6.570275),
 (('B-city_name', 'I-city_name'), 6.388733),
 (('B-arrive_time.end_time', 'I-arrive_time.end_time'), 6.24598),
 (('B-depart_date.month_name', 'B-depart_date.day_number'), 6.240609),
 (('B-depart_time.end_time', 'I-depart_time.end_time'), 6.239169),
 (('B-stoploc.city_name', 'I-stoploc.city_name'), 6.23242)]

### Predictions on Test Data

In [766]:
y_pred = [tagger.tag(xseq) for xseq in X_test]
print(iob_classification_report(y_test, y_pred))

                              precision    recall  f1-score   support

             B-aircraft_code       1.00      0.45      0.62        33
              B-airline_code       0.97      0.85      0.91        34
              B-airline_name       0.99      0.98      0.99       101
              I-airline_name       0.95      0.95      0.95        65
              B-airport_code       0.44      0.44      0.44         9
              B-airport_name       0.88      0.33      0.48        21
              I-airport_name       0.77      0.34      0.48        29
 B-arrive_date.date_relative       0.50      0.50      0.50         2
      B-arrive_date.day_name       0.71      0.45      0.56        11
    B-arrive_date.day_number       0.00      0.00      0.00         6
    B-arrive_date.month_name       0.00      0.00      0.00         6
      B-arrive_time.end_time       0.00      0.00      0.00         8
      I-arrive_time.end_time       0.00      0.00      0.00         8
 B-arrive_time.peri

c:\users\kshitij jain\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Traversing a Chunked Tree

Now that we have labelled (chunked) the validation and test datasets, let's see how we can traverse the trees.

In [767]:
i = random.randrange(len(valid_trees))
chunked_tree = valid_trees[i]

print(' '.join([id_to_words[val] for val in val_x[i]]), '\n')

# traverse the tree and print labels of subtrees 
for n in chunked_tree:
    if isinstance(n, nltk.tree.Tree):
        print(n.label(), n.leaves())

list all tuesday night flights from boston to denver 

depart_date.day_name [('tuesday', 'JJ')]
depart_time.period_of_day [('night', 'NN')]
fromloc.city_name [('boston', 'NN')]
toloc.city_name [('denver', 'VB')]


In [768]:
# correctly parsed complex queries - i=25, 473, 23, 498, 893, 882, 694
# ambiguous queries: not many so far
i

872

In [769]:
# list all labels of train and validation trees

tree_labels = []
for tree in train_trees:
    for n in tree:
        if isinstance(n, nltk.tree.Tree):
            tree_labels.append(n.label())

In [779]:
# training set has 78 unique labels
label_set = set(tree_labels)
len(label_set)

78

In [780]:
list(label_set)

['toloc.airport_code',
 'cost_relative',
 'return_date.day_name',
 'aircraft_code',
 'fare_basis_code',
 'day_number',
 'depart_time.start_time',
 'depart_time.period_mod',
 'return_date.date_relative',
 'arrive_date.date_relative',
 'flight_time',
 'connect',
 'arrive_time.period_mod',
 'meal',
 'stoploc.airport_name',
 'restriction_code',
 'depart_date.day_number',
 'stoploc.city_name',
 'time',
 'round_trip',
 'state_code',
 'days_code',
 'fare_amount',
 'economy',
 'arrive_date.month_name',
 'stoploc.state_code',
 'state_name',
 'depart_time.period_of_day',
 'or',
 'return_date.month_name',
 'fromloc.city_name',
 'meal_description',
 'meal_code',
 'toloc.airport_name',
 'airport_code',
 'mod',
 'depart_time.end_time',
 'depart_date.year',
 'transport_type',
 'flight_number',
 'flight_days',
 'airline_name',
 'today_relative',
 'fromloc.airport_name',
 'day_name',
 'depart_time.time',
 'toloc.state_code',
 'flight_stop',
 'depart_date.today_relative',
 'arrive_time.period_of_day',
 

### FlightStats API Experiments

WIP

In [829]:
# flightstats API experiments
app_id = '9bed5b33'
app_key = 'd7a448569ce9d0821da4fcc9f371e8cc'

base_url = 'https://api.flightstats.com/flex/schedules/rest/v1/json/from/'

# departing from ABQ to DFW on 20 Dec 2018
# {departureAirportCode}/to/{arrivalAirportCode}/departing/{year}/{month}/{day}
extended_url = 'ABQ/to/DFW/departing/2018/12/20'

# credentials
creds = '?appId={0}&appKey={1}'.format(app_id, app_key)

# complete url
url = base_url + extended_url + creds
print(url)

https://api.flightstats.com/flex/schedules/rest/v1/json/from/ABQ/to/DFW/departing/2018/12/20?appId=9bed5b33&appKey=d7a448569ce9d0821da4fcc9f371e8cc


In [830]:
import requests, json
data = requests.get(url).json()

In [835]:
# sample flight details
data['scheduledFlights'][0]

{'arrivalAirportFsCode': 'DFW',
 'arrivalTime': '2018-12-20T09:37:00.000',
 'carrierFsCode': 'AA',
 'codeshares': [{'carrierFsCode': 'JL',
   'flightNumber': '7231',
   'referenceCode': 6844,
   'serviceClasses': ['J', 'Y'],
   'serviceType': 'J',
   'trafficRestrictions': ['Q']},
  {'carrierFsCode': 'BA',
   'flightNumber': '5600',
   'referenceCode': 7010,
   'serviceClasses': ['R', 'J', 'Y'],
   'serviceType': 'J',
   'trafficRestrictions': ['Q']}],
 'departureAirportFsCode': 'ABQ',
 'departureTime': '2018-12-20T06:50:00.000',
 'flightEquipmentIataCode': 'CR9',
 'flightNumber': '5913',
 'isCodeshare': False,
 'isWetlease': True,
 'referenceCode': '477-7022--',
 'serviceClasses': ['R', 'J', 'Y'],
 'serviceType': 'J',
 'stops': 0,
 'trafficRestrictions': [],
 'wetleaseOperatorFsCode': 'YV'}

In [ ]:
# https://developer.flightstats.com/api-docs/scheduledFlights/v1
# https://developer.flightstats.com/api-docs/
# https://developer.flightstats.com/api-docs/how_to